## Data Cleaning

In [258]:
import pandas as pd
import numpy as np
from scipy import stats

In [259]:
athleisure_df = pd.read_csv("athleisure.csv")
# Raw dataframe from API call

athleisure_df.drop(['Unnamed: 0'], axis =1, inplace = True)
# Dropping unneeded column

In [260]:
athleisure_df.head()

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen,month_abbr
0,yoga pants,61,2018,6,google,779,65,12,Jun
1,yoga pants,68,2018,7,google,779,65,12,Jul
2,yoga pants,107,2018,8,google,779,65,12,Aug
3,yoga pants,63,2018,9,google,779,65,12,Sep
4,yoga pants,72,2018,10,google,779,65,12,Oct


In [261]:
athleisure_df.dtypes
# Need to convert year, month to str.

import calendar
athleisure_df['month_abbr'] = athleisure_df.month.apply(lambda x: calendar.month_abbr[x])
athleisure_df['year'] = athleisure_df.year.apply(lambda x: str(x))
athleisure_df['month'] = athleisure_df.month.apply(lambda x: str(x))

In [262]:
# Check volume amounts for Jun months
print(athleisure_df.loc[athleisure_df.month == '6'].volume.unique())
# All are zero. Must impute value

[   61    35   146  1031  1785 69710  7731 10977  9553    11     4     2
    22    74    99    28    59     1     0    14 76026    29  4317 19603
 31552   974   490     7    43    75   381   827  3746 21195    52     9
    10  4979   173  8276    38 -2070    47  3835   601   531    39     3
  3384   112   921   278    13 31618  3343    23   570   948    32   132
    40  2462   206  5628    26  1583   216    44    81   964  2487 18547
  2416   421  8018   228   342  5881    76    37    15  7622 10987   100
   600    62   311  3330   343  7178  3496   868   156   115 11027  1909
   254  2270  4282    21   613  5046  5987   273   193    67  5960 20837
  1082 35410    18    68  1213     6  1963   186 55537  1219 13204    12
    85   192    42  6401    84     8  4149   678   701     5]


## Impute June volume amounts for google searches by subtracting volume totals from given total amount:

In [7]:
def volume_totals():
    # Pandas Series of summed volume totals by keyword
    return athleisure_df.groupby(['keyword','engine']).volume.sum()

def june_volume(row):
    if row.month == '6' and row.volume == 0:
        # Return the difference between the total year volume and the sum of the volume col
        difference = row.total_volume_for_last_12_months - volume_totals()[row.keyword, row.engine]
        return difference
    else:
        return row.volume

# Above only works for google keywords
athleisure_df['volume'] = athleisure_df.apply(june_volume, axis=1)

## Determine which word/engine combinations have faulty June values:

In [8]:
x = athleisure_df[athleisure_df.volume == 0].groupby(['keyword', 'engine']).count()['volume']
y = athleisure_df[athleisure_df.volume == 0].groupby(['keyword', 'engine']).mean()['total_months_seen']
xy = x+y
xy[xy==13]
# If count of zeros in volume col + total months seen = 0, then June data is missing
# 87 values still missing

keyword      engine 
active       amazon     13
             youtube    13
activewear   amazon     13
athletic     amazon     13
             youtube    13
                        ..
windbreaker  amazon     13
workout      amazon     13
             youtube    13
yoga pants   amazon     13
             youtube    13
Length: 87, dtype: int64

## Functions for cleaning remaining columns:

Function for imputing average volume into missing June values:

In [9]:
def june_volume_avg(row):
# Function that imputes the average of 11 months for data still missing from Jun
    for combo in xy[xy==13].index:
        # If count of zeros in volume col + total months seen = 0, then June data is missing
        if row.keyword in combo[0] and row.engine in combo[1] and row.month == '6':
        # Return the average of the other 11 months
    
            new_volume = row.total_volume_for_last_12_months/11
            break
        else:
            new_volume = row.volume
    return int(new_volume)

athleisure_df['volume'] = athleisure_df.apply(june_volume_avg, axis=1)

Function for adding average volume value to incorrect total volume column:

In [10]:
def fix_total_volumes(row):

    for combo in xy[xy==13].index:
        # If count of zeros in volume col + total months seen = 0, then June data is missing
        if row.keyword in combo[0] and row.engine in combo[1]:
            
            new_volume = row.total_volume_for_last_12_months + row.total_volume_for_last_12_months/11
            return int(new_volume)
            break
        else:
            new_volume = row.total_volume_for_last_12_months
    return int(new_volume)

athleisure_df['total_volume_for_last_12_months'] = athleisure_df.apply(fix_total_volumes, axis=1)

Function for correcting avg volume column:

In [11]:
def fix_avg_volumes(row):
    
    for combo in xy[xy==13].index:
        # If count of zeros in volume col + total months seen = 0, then June data is missing
        if row.keyword in combo[0] and row.engine in combo[1]:
    
            new_volume = athleisure_df.loc[(athleisure_df.keyword == row.keyword) & (athleisure_df.month == '6') & (athleisure_df.engine == row.engine)].volume
            return int(new_volume)
            break
        else:
            new_volume = row.avg_volume_for_last_12_months
    return int(new_volume)
            

athleisure_df['avg_volume_for_last_12_months'] = athleisure_df.apply(fix_avg_volumes, axis=1)

In [12]:
# Check if imputed June values match provided avg volume counts
# 71 due to rounding any totals or averages to 1 if zero or <1
(athleisure_df.groupby(['keyword','engine']).volume.mean().round().astype(int) == athleisure_df.groupby(['keyword', 'engine']).avg_volume_for_last_12_months.mean()).value_counts()

True     144
False     71
dtype: int64

In [256]:
# Display DataFrame
athleisure_df.head(15)

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen,month_abbr
0,yoga pants,61,2018,6,google,779,65,12,Jun
1,yoga pants,68,2018,7,google,779,65,12,Jul
2,yoga pants,107,2018,8,google,779,65,12,Aug
3,yoga pants,63,2018,9,google,779,65,12,Sep
4,yoga pants,72,2018,10,google,779,65,12,Oct
5,yoga pants,77,2018,11,google,779,65,12,Nov
6,yoga pants,56,2018,12,google,779,65,12,Dec
7,yoga pants,50,2019,1,google,779,65,12,Jan
8,yoga pants,42,2019,2,google,779,65,12,Feb
9,yoga pants,64,2019,3,google,779,65,12,Mar


In [257]:
athleisure_df.groupby(['keyword','engine']).count().sort_values('volume',ascending=False)
# striped/google has 24 rows and a negative values for June. Need to edit.

volume  year  month  total_volume_for_last_12_months  \
keyword     engine                                                          
striped     google       24    24     24                               24   
3 stripes   google       12    12     12                               12   
split back  amazon       12    12     12                               12   
skinny      amazon       12    12     12                               12   
            google       12    12     12                               12   
...                     ...   ...    ...                              ...   
leggings    google       12    12     12                               12   
            youtube      12    12     12                               12   
lightweight amazon       12    12     12                               12   
            google       12    12     12                               12   
yoga pants  youtube      12    12     12                               12   

                     avg_volume_for_last_12_months  total_months_seen  \
keyword     engine                                                      
striped     google                              24                 24   
3 stripes   google                              12                 12   
split back  amazon                              12                 12   
skinny      amazon                              12                 12   
            google                              12                 12   
...                                            ...                ...   
leggings    google                              12                 12   
            youtube                             12                 12   
lightweight amazon                              12                 12   
            google                              12                 12   
yoga pants  youtube                             12                 12   

                     month_abbr  
keyword     engine               
striped     google           24  
3 stripes   google           12  
split back  amazon           12  
skinny      amazon           12  
            google           12  
...                         ...  
leggings    google           12  
            youtube          12  
lightweight amazon           12  
            google           12  
yoga pants  youtube          12  

[215 rows x 7 columns]

In [269]:
indexes = athleisure_df.loc[(athleisure_df.keyword == 'striped') & (athleisure_df.engine == 'google')][:12].index
# store index values to be deleted
athleisure_df.drop(indexes, inplace = True)
athleisure_df = athleisure_df.reset_index().drop('index',axis=1)

In [303]:
athleisure_df.loc[(athleisure_df.keyword == 'striped') & (athleisure_df.engine == 'google')]

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen,month_abbr
684,striped,95,2018,6,google,2260,188,12,Jun
685,striped,25,2018,7,google,2260,188,12,Jul
686,striped,43,2018,8,google,2260,188,12,Aug
687,striped,76,2018,9,google,2260,188,12,Sep
688,striped,118,2018,10,google,2260,188,12,Oct
689,striped,55,2018,11,google,2260,188,12,Nov
690,striped,151,2018,12,google,2260,188,12,Dec
691,striped,255,2019,1,google,2260,188,12,Jan
692,striped,332,2019,2,google,2260,188,12,Feb
693,striped,404,2019,3,google,2260,188,12,Mar


In [301]:
trace = athleisure_df.loc[(athleisure_df.keyword == 'striped') & (athleisure_df.engine == 'google')]
# Fix volume value for June and avg_volume
athleisure_df.loc[684,'volume'] = 0
athleisure_df.loc[684,'volume'] = 2260 - trace.volume.sum()

In [304]:
# export to csv
athleisure_df.to_csv('athleisure.csv')